In [14]:
#pip install yfinance
#pip install fredapi
#pip install robin_stocks
#pip install ib_insync

SyntaxError: invalid syntax (1315976713.py, line 2)

In [35]:
#Necessary libraries to import
import numpy as np
import pandas as pd
from scipy.stats import norm
import matplotlib.pyplot as plt
import yfinance as yf
from datetime import datetime, timedelta
from fredapi import Fred
import robin_stocks.robinhood as rh


### Black-Scholes Model

The **Black-Scholes model** is a widely-used mathematical model for pricing European-style options. It assumes that the stock price follows a geometric Brownian motion with constant drift and volatility. The model helps determine the theoretical value of call and put options.

The formula for a **call option** price \( C \) is:

$$
C = S \cdot N(d_1) - K \cdot e^{-rT} \cdot N(d_2)
$$

The formula for a **put option** price \( P \) is:

$$
P = K \cdot e^{-rT} \cdot N(-d_2) - S \cdot N(-d_1)
$$

Where:
- \( S \) is the current stock price.
- \( K \) is the option's strike price.
- \( T \) is the time to maturity (in years).
- \( r \) is the risk-free interest rate.
- \( \sigma \) is the volatility of the stock's returns.
- \( N(d_1) \) and \( N(d_2) \) are the cumulative distribution


In [2]:
def black_scholes(S, K, T, r, sigma, option_type='call'):
    # S: Current stock price
    # K: Strike price of the option
    # T: Time to maturity (in years)
    # r: Risk-free interest rate (as a decimal)
    # sigma: Volatility of the stock (standard deviation of returns)
    # option_type: 'call' for call option, 'put' for put option

    # Calculate d1 using the Black-Scholes formula
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    # Calculate d2, which is d1 adjusted for volatility
    d2 = d1 - sigma * np.sqrt(T) 
    # For a call option, calculate the price using the Black-Scholes formula
    if option_type == 'call':
        price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2) 
    # For a put option, calculate the price using the Black-Scholes formula
    elif option_type == 'put':
        price = K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1) 
    # Return the computed option price
    return price

### Monte Carlo Simulation for Option Pricing

The **Monte Carlo method** is a statistical technique used to estimate the price of options by simulating the random paths of the underlying asset price under the risk-neutral measure. The method works by generating a large number of possible future price paths and computing the expected payoff of the option across those paths.

The underlying stock price \( S_t \) at time \( t \) can be modeled using the following stochastic differential equation:

$$
dS_t = rS_t dt + \sigma S_t dW_t
$$

Where:
- \( S_t \) is the stock price at time \( t \),
- \( r \) is the risk-free interest rate,
- \( \sigma \) is the volatility of the stock's returns,
- \( dW_t \) is a Wiener process or Brownian motion representing the random movement of the stock price.

### Steps for the Monte Carlo Simulation:

1. **Simulate Random Paths**:  
   The future stock price \( S_T \) at time \( T \) (the option's maturity) is simulated using the discretized form of the geometric Brownian motion:

   $$
   S_T = S_0 \cdot \exp\left(\left(r - \frac{\sigma^2}{2}\right)T + \sigma \cdot \sqrt{T} \cdot Z\right)
   $$

   Where:
   - \( S_0 \) is the current stock price,
   - \( Z \) is a standard normal random variable \( Z \sim N(0, 1) \).

2. **Estimate Payoff**:  
   For a **call option**, the payoff is calculated as:

   $$
   \text{Payoff}_{\text{call}} = \max(S_T - K, 0)
   $$

   For a **put option**, the payoff is:

   $$
   \text{Payoff}_{\text{put}} = \max(K - S_T, 0)
   $$

   Where \( K \) is the strike price of the option.

3. **Discount Payoffs**:  
   The simulated payoffs are then discounted to present value using the risk-free interest rate:

   $$
   \text{Discounted Payoff} = e^{-rT} \cdot \text{Payoff}
   $$

4. **Compute Option Price**:  
   The option price is the average of all discounted payoffs across the simulated paths:

   $$
   \text{Option Price} = \frac{1}{N} \sum_{i=1}^{N} \text{Discounted Payoff}_i
   $$

   Where \( N \) is the number of simulations.

### Conclusion:

The Monte Carlo method is particularly useful for pricing options with complex features, where closed-form solutions like the Black-Scholes model are not available. However, it may require a large number of simulations for accurate estimates, making it computationally expensive.



In [6]:
def monte_carlo(S, K, T, r, sigma, num_simulations, option_type='call'):
    np.random.seed(42)
    dt = T / num_simulations
    price_matrix = np.zeros((num_simulations, 1))

    for i in range(num_simulations):
        W = np.random.standard_normal(num_simulations)
        S_t = S * np.exp(np.cumsum((r - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * W))
        if option_type == 'call':
            price_matrix[i] = max(0, S_t[-1] - K)
        elif option_type == 'put':
            price_matrix[i] = max(0, K - S_t[-1])

    option_price = np.exp(-r * T) * np.mean(price_matrix)
    return option_price


In [4]:
#Pulls data of the stock price directly from yahoo finance. 
def get_live_stock_price(ticker):
    stock = yf.Ticker(ticker)
    current_price = stock.history(period="1d")['Close'].iloc[-1]
    return current_price

In [7]:
#Get data of option price directly from yahoo finance
def get_option_price(ticker, expiration_date, strike_price, option_type='call'):
    stock = yf.Ticker(ticker)
    options = stock.option_chain(expiration_date)
    
    if option_type == 'call':
        option_data = options.calls
    elif option_type == 'put':
        option_data = options.puts
    
    option = option_data[option_data['strike'] == strike_price]
    
    if not option.empty:
        return option['lastPrice'].values[0]
    else:
        return None

In [8]:
#will get the option expiry data for a ticker. 
def get_option_expiration_dates(ticker):
    stock = yf.Ticker(ticker)
    expiration_dates = stock.options
    return expiration_dates


In [9]:
#How to get strike Price
def get_strike_prices(ticker, expiration_date):
    stock = yf.Ticker(ticker)
    options = stock.option_chain(expiration_date)
    return options.calls['strike'].tolist(), options.puts['strike'].tolist()

In [10]:
# How to get Time to Maturity
def get_time_to_maturity(expiration_date):
    today = datetime.now()
    expiration = datetime.strptime(expiration_date, "%Y-%m-%d")
    days_to_maturity = (expiration - today).days
    return days_to_maturity / 365.0  # Convert days to years


In [11]:
#How to get volatility
def get_historical_volatility(ticker, period="1y"):
    stock = yf.Ticker(ticker)
    data = stock.history(period=period)
    data['Log Returns'] = np.log(data['Close'] / data['Close'].shift(1))
    volatility = data['Log Returns'].std() * np.sqrt(252)  # Annualize the volatility
    return volatility

In [12]:
# Use federal reserve to pull the 10 year treasurt rate, which will be used as the risk free rate for the black scholes model 
fred = Fred(api_key="API_KEY")
risk_free_rate = fred.get_series('DGS10')[-1] / 100  # 10-year Treasury yield
print(f"10-Year Treasury Yield (Risk-Free Rate): {risk_free_rate}")

10-Year Treasury Yield (Risk-Free Rate): 0.0382


C:\Users\omarh\AppData\Local\Temp\ipykernel_11536\3332345628.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  risk_free_rate = fred.get_series('DGS10')[-1] / 100  # 10-year Treasury yield


In [13]:
ticker = 'AAPL'


In [19]:
S = get_live_stock_price(ticker)
K = 250
expiration_date = "2024-12-20"
T = get_time_to_maturity(expiration_date)
r = risk_free_rate  # Manual input or fetch from another source
sigma = get_historical_volatility(ticker)
option_choice = 'call' # Can be 'call' or 'put'

In [20]:
# Get live prices
current_stock_price = get_live_stock_price(ticker)
current_option_price = get_option_price(ticker, expiration_date, K, option_choice)

print(f"Current {ticker} stock price: {current_stock_price}")
print(f"Current {ticker} {option_choice} option price for strike {K} on {expiration_date}: {current_option_price}")
print(f"Current Stock Price (S): {S}")
print(f"Strike Price (K): {K}")
print(f"Time to Maturity (T): {T} years")
print(f"Risk-Free Rate (r): {r}")
print(f"Volatility (sigma): {sigma}")
# Use these values in your Monte Carlo or Black-Scholes models
bs_price = black_scholes(S, K, T, r, sigma, option_choice)
mc_price = monte_carlo(S, K, T, r, sigma, num_simulations=10000, option_type=option_choice)

print(f"Black-Scholes model price: {bs_price}")
print(f"Monte Carlo simulation price: {mc_price}")

Current AAPL stock price: 227.31500244140625
Current AAPL call option price for strike 250 on 2024-12-20: 4.4
Current Stock Price (S): 227.31500244140625
Strike Price (K): 250
Time to Maturity (T): 0.3095890410958904 years
Risk-Free Rate (r): 0.0382
Volatility (sigma): 0.22389778361685747
Black-Scholes model price: 4.4420035199339125
Monte Carlo simulation price: 4.255119480217223


In [21]:
# Function to get viable expiration dates within the next 6 months
def get_viable_expiration_dates(ticker, max_months=6):
    stock = yf.Ticker(ticker)
    all_expiration_dates = stock.options
    today = datetime.now()
    max_date = today + timedelta(days=max_months * 30)
    
    viable_dates = [date for date in all_expiration_dates if today <= datetime.strptime(date, "%Y-%m-%d") <= max_date]
    return viable_dates

# Function to get strike prices within a percentage range of the current stock price
def get_viable_strike_prices(ticker, expiration_date, price_range_percent=10):
    stock = yf.Ticker(ticker)
    current_price = get_live_stock_price(ticker)
    options = stock.option_chain(expiration_date)
    
    min_price = current_price * (1 - price_range_percent / 100)
    max_price = current_price * (1 + price_range_percent / 100)
    
    call_strikes = options.calls['strike']
    put_strikes = options.puts['strike']
    
    viable_strikes = sorted(set(call_strikes[call_strikes.between(min_price, max_price)].tolist() +
                                put_strikes[put_strikes.between(min_price, max_price)].tolist()))
    
    return viable_strikes

In [36]:
def get_robinhood_option_price(ticker, expiration_date, strike_price, option_type='call'):
    # Search for the option ID
    options = rh.options.find_options_by_expiration_and_strike(
        ticker, expirationDate=expiration_date, strikePrice=str(strike_price), optionType=option_type
    )
    
    if options:
        # Fetch market data for the first option found
        option_data = rh.options.get_option_market_data_by_id(options[0]['id'])
        return option_data['adjusted_mark_price']
    else:
        return None

In [41]:
class RobinhoodClient:
    def __init__(self, username, password):
        self.username = username
        self.password = password
        self.login()

    def login(self):
        rh.login(self.username, self.password)

    def get_option_price(self, ticker, expiration_date, strike_price, option_type='call'):
        options = rh.options.find_options_by_expiration_and_strike(
            ticker, expirationDate=expiration_date, strikePrice=str(strike_price), optionType=option_type
        )
        if options:
            option_data = rh.options.get_option_market_data_by_id(options[0]['id'])
            return option_data['adjusted_mark_price']
        else:
            return None
    
    def logout(self):
        rh.logout()

: 

In [38]:
tickers = ["TSLA"]
results = []
# Logging into Robinhood using credentials (replace 'your_username' and 'your_password' with actual credentials)

rh.login('your_username', 'your_password')

# Loop over each stock ticker in the tickers list
for ticker in tickers:
    # Get viable expiration dates for the options (within the next 6 months)
    expiration_dates = get_viable_expiration_dates(ticker, max_months=6)
     # Loop through each expiration date
    for expiration_date in expiration_dates:
        # Get viable strike prices within 20% price range for each expiration date
        strike_prices = get_viable_strike_prices(ticker, expiration_date, price_range_percent=20)
        # Loop through each strike price for the given expiration date
        for strike_price in strike_prices:
            S = get_live_stock_price(ticker) # Get the live/current stock price of the ticker
            T = get_time_to_maturity(expiration_date) # Calculate the time to maturity for the option
            sigma = get_historical_volatility(ticker) # Get historical volatility of the stock price
            
            bs_price = black_scholes(S, strike_price, T, r, sigma, option_type='call') #calculate Black Scholes model
            robinhood_price = get_robinhood_option_price(ticker, expiration_date, strike_price, option_type='call') #Pulls RH price
            # mc_price = monte_carlo(S, K, T, r, sigma, num_simulations=10000, option_type=option_choice) -> would use, but heavy computation resulting in long run times
            results.append({ # Store the result in the results list with all the relevant information
                "Ticker": ticker,
                "Expiration Date": expiration_date,
                "Strike Price": strike_price,
                "Black-Scholes Price": bs_price,
                "Robinhood Price": robinhood_price  
            })

# Print results
for result in results:
    print(result)

Exception: find_options_by_expiration_and_strike can only be called when logged in

In [39]:
# Convert results list to a pandas DataFrame
df_results = pd.DataFrame(results)

# Display the DataFrame
print(df_results)

Empty DataFrame
Columns: []
Index: []
